In [23]:
import pickle
import pandas as pd

In [24]:
import plotly.graph_objects as go
from plotly.offline import plot

In [25]:
# Definimos path
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'

In [26]:
# Definimos colores fundar
fundar_colores_primarios = ['#7BB5C4', '#9FC1AD', '#D3D3E0',  '#8d9bff', '#FF9750', '#FFD900',]
fundar_colores_secundarios = ['#B5E0EA', '#B3B3B3', '#848279', '#AFA36E', '#5D896F', '#9C9CBC', '#E27124']
fundar_white = '#F5F5F5'
fundar_black = '#151515'

In [27]:
with open(pathdata + 'data_output/' + 'diccionario_insercion_laboral_aglomerados.pickle', 'rb') as file:
    dict_dfs_insercion_aglomerado = pickle.load(file)

with open(pathdata + 'data_output/' + 'diccionario_insercion_laboral_urbano.pickle', 'rb') as file:
    dict_dfs_insercion_urbano = pickle.load(file)

df_insercion_pool = pd.read_pickle(pathdata + 'data_output/df_insercion_laboral_pool.pickle')


In [28]:
dates = list(dict_dfs_insercion_aglomerado.keys())
df_temp = dict_dfs_insercion_aglomerado[dates[0]]

# Panel 1: Grafico con 3 barras por provincia: Actividad, Formalidad, Horas Remuneradas

bar_trace1 = go.Bar(
    x=df_temp.index,
    y=df_temp['Actividad'],
    name='Actividad',
    marker_color=fundar_colores_primarios[1],
    hovertemplate='Actividad: %{y:.2f}<extra></extra>'
)

bar_trace2 = go.Bar(
    x=df_temp.index,
    y=df_temp['Formalidad'],
    name='Formalidad',
    marker_color=fundar_colores_primarios[3],
    hovertemplate='Formalidad: %{y:.2f}<extra></extra>'
)

bar_trace3 = go.Bar(
    x=df_temp.index,
    y=df_temp['Horas promedio remuneradas'],
    name='Horas promedio remuneradas',
    marker_color=fundar_colores_primarios[4],
    hovertemplate='Hrs remuneradas: %{y:.2f}<extra></extra>'
)

dropdown_buttons = []
for date in dates:
    df_temp = dict_dfs_insercion_aglomerado[date]
    dropdown_buttons.append(
        dict(
            method='update',
            label=date.strftime('%Y-%m-%d'),
            args=[{'y': [df_temp['Actividad'], df_temp['Formalidad'], df_temp['Horas promedio remuneradas']]}]
        )
    )

updatemenus = [
    dict(
        buttons=dropdown_buttons,
        active=0,
        showactive=True,
        x=0.9,
        xanchor='left',
        y=1.2,
        yanchor='top'
    )
]

layout = go.Layout(
    title='Indicadores inserción laboral',
    barmode='group',
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(x=0.6, y=1.1, orientation='h'),
    updatemenus=updatemenus,
    yaxis_title="Ratio"

)

fig1 = go.Figure(data=[bar_trace1, bar_trace2, bar_trace3], layout=layout)

fig1.add_shape(
    type='line',
    x0=df_temp.index[0],
    y0=100,
    x1=df_temp.index[-1],
    y1=100,
    line=dict(
        color='black',
        width=1,
        dash='dash'
    ),
)
fig1.update_yaxes(range=[0, 120])
file_path = pathdata + 'figs/'
filename = 'panel-1-indicadores.html'
fig1.write_html(file_path + filename)


In [29]:
# Panel 2: Grafico con 1 barra por provincia: Inserción laboral

bar_trace = go.Bar(
    x=df_temp.index,
    y=df_temp['Insercion laboral'],
    name='Insercion laboral',
    marker_color=fundar_colores_primarios[0],
    hovertemplate='Insercion laboral: %{y:.2f}<extra></extra>'
)

dropdown_buttons = []
for date in dates:
    df_temp = dict_dfs_insercion_aglomerado[date]
    dropdown_buttons.append(
        dict(
            method='update',
            label=date.strftime('%Y-%m-%d'),
            args=[{'y': [df_temp['Insercion laboral']]}]
        )
    )

updatemenus = [
    dict(
        buttons=dropdown_buttons,
        active=0,
        showactive=True,
        x=0.9,
        xanchor='left',
        y=1.2,
        yanchor='top'
    )
]

layout = go.Layout(
    title='Componente inserción laboral',
    barmode='group',
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(x=0.6, y=1.1, orientation='h'),
    updatemenus=updatemenus,
    yaxis_title="Ratio"

)

fig2 = go.Figure(data=[bar_trace], layout=layout)

fig2.add_shape(
    type='line',
    x0=df_temp.index[0],
    y0=100,
    x1=df_temp.index[-1],
    y1=100,
    line=dict(
        color='black',
        width=1,
        dash='dash'
    ),
)
fig2.update_yaxes(range=[0, 120])
file_path = pathdata + 'figs/'
filename = 'panel-2-indicadores.html'
fig2.write_html(file_path + filename)


In [30]:
columna_a_extraer = "Insercion laboral"

datos_columnas = {}

for fecha, df in dict_dfs_insercion_aglomerado.items():
    columna = df[columna_a_extraer]
    datos_columnas[fecha] = columna

df_resultado = pd.DataFrame(datos_columnas, index=columna.index).T


In [31]:
# Panel 3: Grafico con 1 linea de la evolucion temporal de la Insercion laboral promediada a nivel nacional

line_trace = go.Scatter(
    x=df_resultado.index,
    y=df_resultado.mean(axis=1),
    name='Insercion laboral',
    marker_color=fundar_black,
    hovertemplate='Insercion laboral: %{y:.2f}<extra></extra>',
    mode='lines+markers',
    marker=dict(
        size=12  
    )
)

layout = go.Layout(
    title='Componente inserción laboral',
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(x=0.6, y=1.1, orientation='h'),
    yaxis_title="Ratio"

)

fig3 = go.Figure(data=[line_trace], layout=layout)
fig3.update_yaxes(range=[82, 85])

file_path = pathdata + 'figs/'
filename = 'panel-3-indicadores.html'
fig3.write_html(file_path + filename)


In [32]:
provincias = list(df_resultado.keys())
df_temp = df_resultado[provincias[0]]

# Panel 4: Grafico con 1 linea de la evolucion temporal de la Insercion laboral para cada provincia
line_trace = go.Scatter(
    x=df_temp.index,
    y=df_temp,
    name='Insercion laboral',
    marker_color=fundar_black,
    hovertemplate='Insercion laboral: %{y:.2f}<extra></extra>',
    mode='lines+markers',
    marker=dict(
        size=12  
    )
)

dropdown_buttons = []
for pcia in provincias:
   df_temp = df_resultado[pcia]
   dropdown_buttons.append(
       dict(
           method='update',
           label=pcia,
            args=[{'y': [df_temp]}]  # Actualizar 'y' con los nuevos datos de cada provincia
       )
   )

updatemenus = [
   dict(
       buttons=dropdown_buttons,
       active=0,
        showactive=True,
        x=0.9,
        xanchor='left',
        y=1.2,
        yanchor='top'
    )
]

layout = go.Layout(
    title='Componente inserción laboral',
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(x=0.6, y=1.1, orientation='h'),
    updatemenus=updatemenus,
    yaxis_title="Ratio"

)

fig4 = go.Figure(data=[line_trace], layout=layout)
fig4.update_yaxes(range=[0, 100])

file_path = pathdata + 'figs/'
filename = 'panel-4-indicadores.html'
fig4.write_html(file_path + filename)


In [52]:
from plotly.subplots import make_subplots

fig5 = make_subplots(rows=2, cols=2,
                    column_widths=[0.7, 0.3],
                    subplot_titles=["Figura 1", "Figura 2", "Figura 3", "Figura 4"])

# Agrega las figuras a las subtramas

for trace in fig1.data:
    fig5.add_trace(trace, row=1, col=1)
for trace in fig2.data:
    fig5.add_trace(trace, row=2, col=1)
for trace in fig3.data:
    fig5.add_trace(trace, row=1, col=2)
for trace in fig4.data:
    fig5.add_trace(trace, row=2, col=2)

fig5.update_layout(
    height=1080,
    width=1920,
    title_text="Dashboard",
    plot_bgcolor='rgba(0,0,0,0)',
    yaxis_title="Ratio"
)
# Menú desplegable para la figura 1
updatemenu_fig1 = go.layout.Updatemenu(
    buttons=list(fig1.layout.updatemenus[0].buttons),
    active=fig1.layout.updatemenus[0].active,
    showactive=fig1.layout.updatemenus[0].showactive,
    x=0.05,
    y=0.95,
    xanchor='left',
    yanchor='top'
)

# Menú desplegable para la figura 2
updatemenu_fig2 = go.layout.Updatemenu(
    buttons=list(fig2.layout.updatemenus[0].buttons),
    active=fig2.layout.updatemenus[0].active,
    showactive=fig2.layout.updatemenus[0].showactive,
    x=0.05,
    y=0.45,
    xanchor='left',
    yanchor='top'
)

# Menú desplegable para la figura 4
updatemenu_fig4 = go.layout.Updatemenu(
    buttons=list(fig4.layout.updatemenus[0].buttons),
    active=fig4.layout.updatemenus[0].active,
    showactive=fig4.layout.updatemenus[0].showactive,
    x=0.55,
    y=0.45,
    xanchor='left',
    yanchor='top'
)

fig5.update_layout(updatemenus=[updatemenu_fig1, updatemenu_fig2, updatemenu_fig4], overwrite=True)

file_path = pathdata + 'figs/'
filename = 'panel-5-indicadores.html'
fig5.write_html(file_path + filename)

ValueError: Can only compare identically-labeled Series objects

In [10]:
import dash
from dash import dcc, html


In [13]:
# Crea la aplicación Dash
app = dash.Dash(__name__)

# Define el layout del dashboard
app.layout = html.Div([
    html.H1("Mi Dashboard"),
    html.Div([
        html.Div([
            dcc.Graph(figure=fig1)
        ], className='six columns'),
        html.Div([
            dcc.Graph(figure=fig2)
        ], className='six columns'),
    ], className='row'),
    html.Div([
        html.Div([
            dcc.Graph(figure=fig3)
        ], className='six columns'),
        html.Div([
            dcc.Graph(figure=fig4)
        ], className='six columns'),
    ], className='row')
])

if __name__ == '__main__':
    app.run_server(

    )


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off
